In [15]:
from flask import Flask, request, render_template
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
import os

# Create the Flask app
app = Flask(__name__)

# Load the processor and model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Define the home route
@app.route('/')
def home():
    return render_template('index.html')

# Define the caption route
@app.route('/caption', methods=['POST'])
def caption():
    if 'file' not in request.files:
        return 'No file part'
    file = request.files['file']
    if file.filename == '':
        return 'No selected file'

    # Process the image
    image = Image.open(file.stream)
    inputs = processor(images=image, return_tensors="pt")

    # Generate caption
    with torch.no_grad():
        outputs = model.generate(**inputs)
    caption = processor.decode(outputs[0], skip_special_tokens=True)

    return f'''
        <h1>Generated Caption</h1>
        <p>{caption}</p>
        <a href="/">Go Back</a>
    '''

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [23/Oct/2024 14:50:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2024 14:50:48] "POST /caption HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2024 14:50:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2024 14:51:12] "POST /caption HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2024 14:51:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2024 14:54:39] "POST /caption HTTP/1.1" 200 -


In [6]:
from flask import Flask, request, render_template, jsonify
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
import base64
from io import BytesIO

# Create the Flask app
app = Flask(__name__)

# Load the processor and model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Define the home route
@app.route('/')
def home():
    return render_template('index.html')

# Define the caption route
@app.route('/caption', methods=['POST'])
def caption():
    if 'file' in request.files:  # For file uploads
        file = request.files['file']
        if file.filename == '':
            return jsonify({"error": "No selected file"}), 400
        
        # Process the image from file upload
        image = Image.open(file.stream)
        
    elif request.is_json:  # For JSON data from camera capture
        data = request.get_json()
        image_data = data.get('image')
        if not image_data:
            return jsonify({"error": "No image provided"}), 400

        # Decode the base64 image
        image_data = image_data.split(',')[1]  # Remove the data:image/png;base64, part
        image = Image.open(BytesIO(base64.b64decode(image_data)))
        
    else:
        return jsonify({"error": "Invalid request format"}), 400

    # Generate caption
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs)
    caption = processor.decode(outputs[0], skip_special_tokens=True)

    return jsonify({"caption": caption})

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Oct/2024 14:22:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 14:22:58] "GET /static/camera.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 14:23:21] "POST /caption HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 14:23:52] "POST /caption HTTP/1.1" 200 -


In [ ]:
from flask import Flask, request, render_template, jsonify
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
import base64
from io import BytesIO

# Create the Flask app
app = Flask(__name__)

# Load the processor and model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Define the home route
@app.route('/')
def home():
    return render_template('index.html')

# Define the caption route
@app.route('/caption', methods=['POST'])
def caption():
    if 'file' in request.files:  # For file uploads
        file = request.files['file']
        if file.filename == '':
            return jsonify({"error": "No selected file"}), 400
        
        # Process the image from file upload
        image = Image.open(file.stream).convert("RGB")
        
    elif request.is_json:  # For JSON data from camera capture
        data = request.get_json()
        image_data = data.get('image')
        if not image_data:
            return jsonify({"error": "No image provided"}), 400

        # Decode the base64 image
        image_data = image_data.split(',')[1]  # Remove the data:image/png;base64, part
        image = Image.open(BytesIO(base64.b64decode(image_data))).convert("RGB")
        
    else:
        return jsonify({"error": "Invalid request format"}), 400

    # Generate caption
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs)
    caption = processor.decode(outputs[0], skip_special_tokens=True)

    return jsonify({"caption": caption})

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Oct/2024 14:34:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 14:34:01] "GET /static/camera.js HTTP/1.1" 304 -
127.0.0.1 - - [26/Oct/2024 14:34:18] "POST /caption HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 14:34:43] "POST /caption HTTP/1.1" 200 -
